## BERT as a service

This notebook demonstrates how to build a complete machine learning pipeline using TensorFlow Extended ([TFX](https://www.tensorflow.org/tfx)) to serve a BERT model for text sentiment classification.

Notes:
 - Data: IMDB Movie Reviews (5000 samples) [original source](https://ai.stanford.edu/~amaas/data/sentiment/)
 - Model: BERT base uncased (english) from [HuggingFace](https://huggingface.co/bert-base-uncased)
 - Processor: BERT uncased (english seq_length=128) from [TF HUB](https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3)


You can check references, additional information, and resources at the [GitHub repository](https://github.com/dimitreOliveira/bert-as-a-service_TFX).

## Setup

In [1]:
# Kubleflow setup
import sys
# Use the latest version of pip.
!pip install --upgrade -q pip
# Install tfx and kfp Python packages.
!pip install -q --pre tfx[kfp]==1.0.0rc1

# # General setup
!pip install -q tensorflow_text

### Import packages

In [2]:
# import os
# import json
# import absl
# import shutil
# import pprint
import urllib
# import tempfile
# import requests
import tensorflow as tf
import tfx
# import grpc
# import transformers

In [3]:
# # Set up logging.
# absl.logging.set_verbosity(absl.logging.INFO)

# tf.get_logger().propagate = False
# pp = pprint.PrettyPrinter()
# %load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [4]:
print(f'TensorFlow version: {tf.__version__}')
print(f'TFX version: {tfx.__version__}')

TensorFlow version: 2.5.0
TFX version: 1.0.0-rc1


[Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/) environment variables

In [5]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GOOGLE_CLOUD_PROJECT=shell_output[0]
%env GOOGLE_CLOUD_PROJECT={GOOGLE_CLOUD_PROJECT}
print("GCP project ID:" + GOOGLE_CLOUD_PROJECT)

env: GOOGLE_CLOUD_PROJECT=tfx-bert-aas
GCP project ID:tfx-bert-aas


KFP endpoint
"AI Platform > Pipeline > pipeline instance `settings`"
**ENDPOINT should contain only the hostname part of the URL.** For example, if the URL of the KFP dashboard is `https://1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com/#/start`, ENDPOINT value becomes `1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com`.

In [6]:
# This refers to the KFP cluster endpoint
ENDPOINT='3ad82eac8d7dd4b7-dot-us-central1.pipelines.googleusercontent.com'
if not ENDPOINT:
    from absl import logging
    logging.error('Set your ENDPOINT in this cell.')

Set the image name as `tfx-pipeline` under the current GCP project.

In [7]:
# Docker image name for the pipeline image.
CUSTOM_TFX_IMAGE='gcr.io/' + GOOGLE_CLOUD_PROJECT + '/tfx-pipeline'
print(CUSTOM_TFX_IMAGE)

gcr.io/tfx-bert-aas/tfx-pipeline


## Step 2. Copy the predefined template to your project directory.

In this step, we will create a working pipeline project directory and files by copying additional files from a predefined template.

You may give your pipeline a different name by changing the `PIPELINE_NAME` below. This will also become the name of the project directory where your files will be put.

In [8]:
PIPELINE_NAME="my_pipeline"
import os
PROJECT_DIR=os.path.join(os.path.expanduser("~"),"imported",PIPELINE_NAME)

Change the working directory context in this notebook to the project directory.

In [9]:
%cd {PROJECT_DIR}

/home/jupyter/imported/my_pipeline


## Source files

Here is brief introduction to each of the Python files.
-   `data` - This directory contains the datasets.
-   `models` - This directory contains ML model definitions.
    -   `bert_aas_utils.py` — defines utility functions for the pipeline
-   `pipeline` - This directory contains the definition of the pipeline
    -   `configs.py` — defines common constants for pipeline runners
    -   `pipeline.py` — defines TFX components and a pipeline
-   `local_runner.py`, `kubeflow_runner.py` — define runners for each orchestration engine


### Download data

In [ ]:
data_csv_url = 'https://raw.githubusercontent.com/dimitreOliveira/bert-as-a-service_TFX/main/Data/IMDB_5k_dataset.csv'
# data_txt_url = 'https://raw.githubusercontent.com/dimitreOliveira/bert-as-a-service_TFX/main/Data/IMDB_5k_dataset.csv'
data_csv_filename = 'IMDB_dataset.csv'
# data_txt_filename = 'IMDB_dataset.txt'

_data_dir = 'data/'

# Download data
urllib.request.urlretrieve(data_csv_url, f'{_data_dir}{data_csv_filename}')
# urllib.request.urlretrieve(data_txt_url, f'{_data_dir}{data_txt_filename}')

Upload our sample data to GCS bucket so that we can use it in our pipeline later.

In [ ]:
!gsutil cp data/IMDB_dataset.csv gs://{GOOGLE_CLOUD_PROJECT}-kubeflowpipelines-default/bert-aas/data/IMDB_dataset.csv

## Create TFX pipeline

Components in the TFX pipeline will generate outputs for each run as [ML Metadata Artifacts](https://www.tensorflow.org/tfx/guide/mlmd), and they need to be stored somewhere. You can use any storage which the KFP cluster can access, and for this example we will use Google Cloud Storage (GCS). A default GCS bucket should have been created automatically. Its name will be `<your-project-id>-kubeflowpipelines-default`.

Let's create a TFX pipeline using the `tfx pipeline create` command.

>Note: When creating a pipeline for KFP, we need a container image which will be used to run our pipeline. And `skaffold` will build the image for us. Because skaffold pulls base images from the docker hub, it will take 5~10 minutes when we build the image for the first time, but it will take much less time from the second build.

In [10]:
!tfx pipeline create  \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT} \
--build-image

CLI
Creating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
/opt/conda/lib/python3.7/site-packages/kfp/_client.py:182: UserWarning: The host 3ad82eac8d7dd4b7-dot-us-central1.pipelines.googleusercontent.com does not contain the "http" or "https" protocol. Defaults to "https".
  ' Defaults to "https".' % host)
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
[Docker] Step 1/4 : FROM tensorflow/tfx:1.0.0-rc1[Docker] 
[Docker] The push refers to repository [gcr.io/tfx-bert-aas/my_pipeline]
New container image "gcr.io/tfx-bert-aas/my_pipeline@sha256:a040ebe64479b934e919395dc6fbc3d353293643a696381d37f8585c6eed85cd" was built.
INFO:absl:Generating ephemeral wheel package for '/home/jupyter/imported/my_pipeline/pipeline/bert_aas_utils.py' (including modules: ['bert_aas_utils', 'configs', 'pipeline']).
INF

While creating a pipeline, `Dockerfile` will be generated to build a Docker image. Don't forget to add it to the source control system (for example, git) along with other source files.

NOTE: `kubeflow` will be automatically selected as an orchestration engine if `airflow` is not installed and `--engine` is not specified.

## Run TFX pipeline

Now start an execution run with the newly created pipeline using the `tfx run create` command.

In [11]:
!tfx run create  \
--pipeline-name={PIPELINE_NAME} \
--endpoint={ENDPOINT}

CLI
Creating a run for pipeline: my_pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
/opt/conda/lib/python3.7/site-packages/kfp/_client.py:182: UserWarning: The host 3ad82eac8d7dd4b7-dot-us-central1.pipelines.googleusercontent.com does not contain the "http" or "https" protocol. Defaults to "https".
  ' Defaults to "https".' % host)
Run created for pipeline: my_pipeline
+===============+======================================+========+===========================+=============================================================================================================================+
| pipeline_name | run_id                               | status | created_at                | link                                                                                                                        |
+===============+======================================+========+===========================+==================================================

Or, you can also run the pipeline in the KFP Dashboard.  The new execution run will be listed under Experiments in the KFP Dashboard.  Clicking into the experiment will allow you to monitor progress and visualize the artifacts created during the execution run.

However, we recommend visiting the KFP Dashboard. You can access the KFP Dashboard from the Cloud AI Platform Pipelines menu in Google Cloud Console. Once you visit the dashboard, you will be able to find the pipeline, and access a wealth of information about the pipeline.
For example, you can find your runs under the *Experiments* menu, and when you open your execution run under Experiments you can find all your artifacts from the pipeline under *Artifacts* menu.

>Note: If your pipeline run fails, you can see detailed logs for each TFX component in the Experiments tab in the KFP Dashboard.
    
One of the major sources of failure is permission related problems. Please make sure your KFP cluster has permissions to access Google Cloud APIs. This can be configured [when you create a KFP cluster in GCP](https://cloud.google.com/ai-platform/pipelines/docs/setting-up), or see [Troubleshooting document in GCP](https://cloud.google.com/ai-platform/pipelines/docs/troubleshooting).

## Step 5. Add components for data validation.

In this step, you will add components for data validation including `StatisticsGen`, `SchemaGen`, and `ExampleValidator`. If you are interested in data validation, please see [Get started with Tensorflow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

>**Double-click to change directory to `pipeline` and double-click again to open `pipeline.py`**. Find and uncomment the 3 lines which add `StatisticsGen`, `SchemaGen`, and `ExampleValidator` to the pipeline. (Tip: search for comments containing `TODO(step 5):`).  Make sure to save `pipeline.py` after you edit it.

You now need to update the existing pipeline with modified pipeline definition. Use the `tfx pipeline update` command to update your pipeline, followed by the `tfx run create` command to create a new execution run of your updated pipeline.


In [14]:
# Update the pipeline
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
# You can run the pipeline the same way.
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

CLI
Updating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
/opt/conda/lib/python3.7/site-packages/kfp/_client.py:182: UserWarning: The host 3ad82eac8d7dd4b7-dot-us-central1.pipelines.googleusercontent.com does not contain the "http" or "https" protocol. Defaults to "https".
  ' Defaults to "https".' % host)
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Generating ephemeral wheel package for '/home/jupyter/imported/my_pipeline/pipeline/bert_aas_utils.py' (including modules: ['bert_aas_utils', 'configs', 'pipeline']).
INFO:absl:User module package has hash fingerprint version 6796558132309e31778e303a5673f4de4b58135471a6dbc29095acca50bd0590.
INFO:absl:Executing: ['/opt/conda/bin/python3.7', '/tmp/tmpcehcxft8/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpatsemeon', '--

### Check pipeline outputs

Visit the KFP dashboard to find pipeline outputs in the page for your pipeline run. Click the *Experiments* tab on the left, and *All runs* in the Experiments page. You should be able to find the latest run under the name of your pipeline.

When this execution run finishes successfully, you have now created and run your first TFX pipeline in AI Platform Pipelines!

**NOTE:** If we changed anything in the model code, we have to rebuild the
container image, too. We can trigger rebuild using `--build-image` flag in the
`pipeline update` command.

**NOTE:** You might have noticed that every time we create a pipeline run, every component runs again and again even though the input and the parameters were not changed.
It is waste of time and resources, and you can skip those executions with pipeline caching. You can enable caching by specifying `enable_cache=True` for the `Pipeline` object in `pipeline.py`.


## Step 9. (*Optional*) Try Cloud AI Platform Training and Prediction with KFP

TFX interoperates with several managed GCP services, such as [Cloud AI Platform for Training and Prediction](https://cloud.google.com/ai-platform/). You can set your `Trainer` component to use Cloud AI Platform Training, a managed service for training ML models. Moreover, when your model is built and ready to be served, you can *push* your model to Cloud AI Platform Prediction for serving. In this step, we will set our `Trainer` and `Pusher` component to use Cloud AI Platform services.

>Before editing files, you might first have to enable *AI Platform Training & Prediction API*.

>**Double-click `pipeline` to change directory, and double-click to open `configs.py`**. Uncomment the definition of `GOOGLE_CLOUD_REGION`, `GCP_AI_PLATFORM_TRAINING_ARGS` and `GCP_AI_PLATFORM_SERVING_ARGS`. We will use our custom built container image to train a model in Cloud AI Platform Training, so we should set `masterConfig.imageUri` in `GCP_AI_PLATFORM_TRAINING_ARGS` to the same value as `CUSTOM_TFX_IMAGE` above.

>**Change directory one level up, and double-click to open `kubeflow_runner.py`**. Uncomment `ai_platform_training_args` and `ai_platform_serving_args`.

Update the pipeline and create an execution run as we did in step 5 and 6.

In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

You can find your training jobs in [Cloud AI Platform Jobs](https://console.cloud.google.com/ai-platform/jobs). If your pipeline completed successfully, you can find your model in [Cloud AI Platform Models](https://console.cloud.google.com/ai-platform/models).

## Step 10. Ingest YOUR data to the pipeline

We made a pipeline for a model using the Chicago Taxi dataset. Now it's time to put your data into the pipeline.

Your data can be stored anywhere your pipeline can access, including GCS, or BigQuery. You will need to modify the pipeline definition to access your data.

1. If your data is stored in files, modify the `DATA_PATH` in `kubeflow_runner.py` or `local_runner.py` and set it to the location of your files. If your data is stored in BigQuery, modify `BIG_QUERY_QUERY` in `pipeline/configs.py` to correctly query for your data.
1. Add features in `models/features.py`.
1. Modify `models/preprocessing.py` to [transform input data for training](https://www.tensorflow.org/tfx/guide/transform).
1. Modify `models/keras/model.py` and `models/keras/constants.py` to [describe your ML model](https://www.tensorflow.org/tfx/guide/trainer).
  - You can use an estimator based model, too. Change `RUN_FN` constant to `models.estimator.model.run_fn` in `pipeline/configs.py`.

Please see [Trainer component guide](https://www.tensorflow.org/tfx/guide/trainer) for more introduction.

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Alternatively, you can clean up individual resources by visiting each consoles:
- [Google Cloud Storage](https://console.cloud.google.com/storage)
- [Google Container Registry](https://console.cloud.google.com/gcr)
- [Google Kubernetes Engine](https://console.cloud.google.com/kubernetes)
